In [1]:
#Import Required Packages
import json
import os

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score

Using TensorFlow backend.


In [2]:
#Define File Paths as Constants
TWITS_PATH = r"../stocktwits_prediction/Data/twits"
FINANCE_PATH = r"../stocktwits_prediction/Data/csv"

#Define Company Names and List of Tags as Constant
COMPANY_NAMES = ["apple","boeing", "caterpillar", "cisco", "chevron", "dupont", "de", "nemours", "walt", 
                 "diseny", "facebook", "google", "vaneck", "goldman", "sachs", "ibm", "intel", "johnson",
                "jpmorgan", "coca", "cola", "coca-cola", "mcdonalds", "3m", "merck", "microsoft", "nike",
                "pfizer", "unitedhealth", "raytheon", "visa", "verizon", "walmart", 
                 "exxon", "mobil"]

TAGS = ['\$FB','\$GOOG','\$AAPL', '\$WB','\$AXP','\$BA','\$CAT','\$CSCO','\$CVX','\$DD','\$DIS','\$FB',
        '\$GE','\$GS','\$HD','\$IBM','\$INTC','\$JNJ','\$JPM','\$KO','\$MCD','\$MMM','\$MRK','\$MSFT',
        '\$NKE','\$PFE','\$PG','\$QQQ','\$SPY','\$TRV','\$UNH','\$UTX','\$V','\$VZ','\$WMT','\$XOM']

LABELS = ["Positive Return", "Negative Return"]

#Return ALl Files in Directory
all_twitter = os.listdir(TWITS_PATH)
all_finance = os.listdir(FINANCE_PATH)

#Generate List of Tags
all_tags = list()
for file_name in all_twitter:
    tag = '$' + file_name[:-4]
    all_tags.append(tag)

# stock_twits_metrics
###  - Inputs:
        - raw_data: DataFrame (can use returned DataFrame from the text parser
        - n (int): number of days (used for s_t calculation)
        - file_name (str): name of file - ex. 'FB.txt'
###  - Returns Dataframe with Following Value Types:
        - Date: DateTime object representing the a specific date in the sample period
        - bearish: total count of tweets tagged with bearish sentiment on that day
        - bullish: total count of tweets tagged with bullish sentiment on that day
        - none: total count of tweets tagged with neither bearish nor bullisn sentiment on that day
        - message_volume: total number of messages on that day 
        - mv1_t:  percentage change in message volume with one day period
        - mv10_t: percentage change in message volume with ten day period
        - polarity: (bullish - bearish)/total messages on that day
        - s_t: moving average of polarity over n days
        - company: string containing the abbreviation of the company name
###  - Additional Notes:
        - function aggregates the data into values by date - does not include the text of the orignal tweets in dataset

In [3]:
def stock_twits_metrics(raw_data, n, file_name):
    #Unwrap Sentiment Values to New Columns
    raw_data['sentiment'] = raw_data['sentiment'].apply(lambda x:{} if pd.isna(x) else x)
    sentiment_vals = json_normalize(data=raw_data['sentiment'], meta=['class','name'])
    sentiment_vals = sentiment_vals.drop(columns='name')
    sentiment_vals = sentiment_vals.replace(np.nan, 'none', regex=True)

    #Convert Date Strings to DateTime Objects
    raw_data['created_at'] = pd.to_datetime(raw_data['created_at']).apply(lambda x: x.date())

    #Create New Dataframe
    temp = pd.concat([raw_data['body'], sentiment_vals, raw_data['created_at']],axis=1)
    
    #Aggregate Text From Tweets By Date
    agg_text = temp.groupby('created_at')['body'].apply(lambda x: x.sum())
    agg_text.to_frame()
    
    #Group by Unique Dates and Extract Value Counts
    message_volume = temp['created_at'].value_counts().rename_axis('dates').reset_index(name='message_volume')
    message_volume = message_volume.sort_values(by='dates').reset_index(drop=True)

    #Create Pivot Table With Desired Statistics
    temp['count'] = 1
    table = temp.pivot_table(index=['created_at'], dropna=False,
                             columns='class',values='count',
                             fill_value=0,aggfunc=np.sum)
    
    #Merge Table with Message Volume
    table = table.reset_index()
    table = pd.concat([table,message_volume['message_volume']],axis=1)

    #Calculate Metrics and Append to the DataFrame
    table['mv1_t'] = table['message_volume'].diff(periods=1).div(table['message_volume'].shift(1))
    table['mv10_t'] = table['message_volume'].div(table['message_volume'].rolling(10).mean())

    table['polarity'] = table.apply(lambda date: (date.bullish - date.bearish)/date.message_volume, axis = 1)
    table['s_t'] = table['polarity'].rolling(n).mean()
    
    #Append Company Name to DataFrame
    name = file_name[:-4]
    table['company'] = name
    
    #Append Text to DataFrame
    result = pd.merge(table, agg_text, on='created_at')
    
    #Change Column Name to Date
    result = result.rename(columns={"created_at":"Date"})
    
    return result

# stock_twits_text_parser
###  - Inputs:
        - file (str): name of file - ex. 'FB.txt'
###  - Returns Dataframe with Following Value Types:
        - df: DataFrame with all rows containing multiple tagged companies dropped, all columns included - index reset to reflect new size
###  - Additional Notes:
        - None

In [4]:
def stock_twits_text_parser(file):
    #Read File
    file_path = TWITS_PATH + '/' + file
    raw_data = pd.read_json(file_path)

    #Create New Column with List of Unique Tags
    raw_data["instances"] = (raw_data['body'].str.findall(f'(?i)({"|".join(TAGS)})')
                                .apply(lambda x: list(set(map(str.upper,x))))
                            )
    #Drop All Rows With Multiple Tags and Reset Index
    df = raw_data[raw_data['instances'].map(lambda x: len(x)) < 2]
    df = df.reset_index()
    
    return df

# feature_selector
###  - Inputs:
        - metrics (DataFrame): DataFrame containing the body text of the tweet eg. data['body'] along with other metrics
        - labels (DataFrame) DataFrame containing the target labels
        - target_label (str): name of the column containing the target label we want to use
        - min_count(int) : minimum number of occurances so that the word is converted
        - n (int): number of text features selected
###  - Returns Dataframe with Following Value Types:
        - df: DataFrame with top n text features and all numerical features, raw data and labels (20 + n columns total)
###  - Additional Notes:
        - text features will always be appended after the numerical metrics, raw data and labels - currently these are the first 20 columns: use df.iloc[:,20:] to extract only text features for analysis
        - text features selected using chi squared statistics
        - ***TODO: Currently the method selects features based on categorical outcomes (classification problem) this will need to be changed to allow for regression (explore a way to bin continuous target data) ***

In [5]:
def feature_selector(metrics, labels, target_label, min_count, n):
    #Merge DataFranes
    result = pd.merge(metrics, labels, on='Date')
    
    #Remove Company Names By Adding Them To Stop Words
    new_stop_words = text.ENGLISH_STOP_WORDS.union(COMPANY_NAMES)

    #Vectorizer Parameters: Convert To Lowercase, Remove Stop Words With Company Names,Min Count 25, Laplace Smoothing
    v = TfidfVectorizer(analyzer='word', lowercase=True,stop_words=new_stop_words, min_df = min_count, smooth_idf=True)
    x = v.fit_transform(result['body'])

    #Convert back to DataFrame
    text_data = pd.DataFrame(x.toarray(),columns=v.get_feature_names())
    
    #Select n Best Features
    X = text_data
    Y = convert_to_binary_classification(result[target_label])
    chi2_selector = SelectKBest(chi2, k=n)
    X_best = chi2_selector.fit_transform(X,Y)
    
    data = pd.DataFrame(X_best)
    df = pd.concat([result,data],axis=1)
    
    return df

# return_over_period_T
###  - Inputs:
        - file (str): name of file - ex. 'FB.csv'
        - T (int): period based on which return is calculated (in days)
###  - Returns Dataframe with Following Value Types:
        - Date: DateTime object representing a specific date in the sample period
        - OPEN, HIGH, LOW, VOLUME, CLOSE: prices each day with respect to the specific metric
        - xxx_return: return for referenced metric over period T, where T is specified when the function is called
###  - Additional Notes:
        - function includes original data taken from the CSV file
        - formula for return over period T is defined as: r(T) = (p(t+T) - p(t))/p(t), where t is current day and p(t) i the price at the current day

In [6]:
def return_over_period_T(file, T):
    #Read File
    file_path = FINANCE_PATH + '/' + file
    df = pd.read_csv(file_path)

    #Convert Date Strings to DateTime Objects
    df['Date'] = pd.to_datetime(df['Date']).apply(lambda x: x.date())

    #Calculate Metrics for OPEN, HIGH, LOW, VOLUME and CLOSE
    df['open_return'] = -df['OPEN'].diff(periods = -T).div(df['OPEN'])
    df['high_return'] = -df['HIGH'].diff(periods = -T).div(df['HIGH'])
    df['low_return'] = -df['LOW'].diff(periods = -T).div(df['LOW'])
    df['close_return'] = -df['CLOSE'].diff(periods = -T).div(df['CLOSE'])
    
    df = df.dropna()
    
    return df

# convert_to_binary_classification
###  - Inputs:
        - df (DataFrame): single column to be classified
###  - Returns Dataframe with Following Value Types:
        - Binary Labels: single column of binary labels representing positive and negative returns on the day
###  - Additional Notes:
        - Note that a negative return is defined as class 0 and a postive return or no return is defined as class 1

In [7]:
def convert_to_binary_classification(df):
    binary_classification = df.apply(lambda x: 0 if x < 0 else 1)
    return binary_classification

# split_data
###  - Inputs:
        - df (DataFrame): Processed DataFrame with features and labels to be split
        - train_percentage (float): Percentage of data to be used as training data - assumes all other data is used as test
        - features (str): toggles whether to use text only or text and numerical features (valid inputs: 'text', 'all')
        - label (str): selects target label based on column name
        - target_type (str): binary (bin) or continuous (cont)
###  - Returns Dataframe with Following Value Types:
        - data_dict (Dictionary): dictionary with 4 values corresponding to X_train, X_test, Y_train, Y_test (keys)
###  - Additional Notes:
        - ***TODO: does not handle continous data correctly***

In [8]:
def split_data(df, train_percentage, features, label, target_type):
    #Determine Split Index Based on the Percentage Of Data to Be Used For Training
    train_index = int(train_percentage*len(df.index)) + 1
    
    #Split Data
    if(features == 'text'):
        X_train = df.iloc[0:train_index,20:]
        X_test = df.iloc[train_index:,20:]
    elif(features == 'all'):
        df1 = df.iloc[0:train_index:,np.r_[5:7,8]]
        df2 = df.iloc[0:train_index,20:]
        X_train = pd.concat([df1,df2],axis=1)
        
        df3 = df.iloc[train_index:,np.r_[5:7,8]]
        df4 = df.iloc[train_index:,20:]
        X_test = pd.concat([df3,df4],axis=1)
    else:
        raise ValueError('Invalid Parameter Input - features')
        
    if(target_type == 'bin' or target_type == 'cont'):
        if(target_type == 'bin'):
            Y = convert_to_binary_classification(df[label])
        Y_train = Y.iloc[0:train_index]
        Y_test = Y.iloc[train_index:]
    else:
        raise ValueError('Invalid Parameter Input - target_type')
    
    #Return as a Dictionary
    data_dict = dict()
    
    data_dict['X_train'] = X_train
    data_dict['X_test'] = X_test
    data_dict['Y_train'] = Y_train
    data_dict['Y_test'] = Y_test
    
    return data_dict

# create_metrics_table
###  - Inputs:
        - Y_train: training labels
        - pred_train: predicted labels from training data
        - Y_test: training labels
        - pred_test: predicted labels from test data
        - valid input types (series, numpy array, DF columns) - must be column vectors
###  - Returns Dataframe with Following Value Types:
        - table (DataFrame): table with REC, PREC, F1 amd ACC for training and test predictions
###  - Additional Notes:
        - None

In [9]:
def create_metrics_table(Y_train, pred_train, Y_test, pred_test):
    #Calculate Metrics
    REC_train = recall_score(Y_train,pred_train)
    PREC_train = precision_score(Y_train, pred_train)
    F1_train = f1_score(Y_train, pred_train)
    ACC_train = accuracy_score(Y_train, pred_train)

    REC_test = recall_score(Y_test,pred_test)
    PREC_test = precision_score(Y_test,pred_test)
    F1_test = f1_score(Y_test, pred_test)
    ACC_test = accuracy_score(Y_test, pred_test)

    #Create Table
    rows = [["Training", REC_train, PREC_train, F1_train, ACC_train], ["Test", REC_test, PREC_test, F1_test,ACC_test]]
    table = pd.DataFrame(rows, columns = ["Dataset", "Recall", "Precision", "F1 Score", "Accuracy"])
    table.set_index("Dataset", inplace =True)

    return table

# plot_confusion_matrix
###  - Inputs:
        - matrix (confusion matrix): confusion matrix generated from sklearn confusion_matrix
        - data_set_name (str): name of dataset ex. 'training' or 'test' 
###  - Returns Dataframe with Following Value Types:
        - None - void type function
        - Displays plot
###  - Additional Notes:
        - Wrapper for seaborn plot code

In [10]:
def plot_confusion_matrix(matrix, data_set_name):
    sns.heatmap(matrix, annot=True, fmt= ".3f", xticklabels = LABELS, yticklabels = LABELS)
    plt.xlabel("Predicted Label")
    plt.ylabel("Actual Label")
    plt.title("Confusion Matrix for " + data_set_name + " Set")

# plot_ROC_curve
###  - Inputs:
        - FPR: False Positive Rate - Generated from sklearn roc_curve
        - TPR: True Positive Rate - Generated from sklearn roc_curve
        - data_set_name (str): name of dataset ex. 'training' or 'test'
###  - Returns Dataframe with Following Value Types:
        - None - void type function
        - Displays plot
###  - Additional Notes:
        - Wrapper for matplotlib plot code

In [11]:
def plot_ROC_curve(FPR, TPR, data_set_name):
    plt.plot([0,1],[0,1],'k--')
    plt.plot(FPR, TPR)
    plt.xlabel("False Positive")
    plt.ylabel("True Positive")
    plt.title("ROC Curve for " + data_set_name + " Data")
    plt.show()

# get_best_SVC
###  - Inputs:
        - model_type (str): toggles between selecting the best linear model or choosing from all types - valid inputs ('all', 'linear')
        - alphas (list): list containing the distinct values of the regularization parameter to be chosen between
        - folds (int): number of folds used for cross validation
        - X_train (DataFrame): training feature data
        - Y_train (DataFrame): training label data
###  - Returns Dataframe with Following Value Types:
        - best_model (estimator object): returns estimator selected based on the given parameters
###  - Additional Notes:
        - prints the parameters selected to generate the best model

In [12]:
def get_best_SVC(model_type, alphas, folds, X_train, Y_train):
    if(model_type == 'linear'):
        lin_svc = LinearSVC(dual=False ,max_iter=500000)
        params = {'penalty':['l1','l2'],'C':alphas}
        clf = GridSearchCV(estimator=lin_svc, param_grid=params, cv=folds)
        clf.fit(X_train,Y_train)
        
        print("Best Parameters: ")
        print(clf.best_params_)
        
        best_model = clf.best_estimator_
    elif(model_type == 'all'):
        svc = SVC(max_iter=500000)
        params = {'C':alphas, 'kernel':['linear','poly','sigmoid','rbf']}
        clf = GridSearchCV(estimator=svc, param_grid=params, cv=folds)
        clf.fit(X_train,Y_train)
        
        print("Best Parameters: ")
        print(clf.best_params_)
        
        best_model = clf.best_estimator_
    else:
        raise ValueError('Invalid Parameter Input - model_type')
    return best_model

# Testing

In [ ]:
#file_name = 'FB.txt'
#csv_name = 'FB.csv'

#company = stock_twits_text_parser(file_name)
#company_metrics = stock_twits_metrics(company, 3, file_name)
#company_return = return_over_period_T(csv_name,3)
#agg_data = feature_selector(company_metrics, company_return, 'close_return', 25, 1000)

In [14]:
#company_data = split_data(agg_data, 0.7, 'all', 'close_return', 'bin')